### Multivariate Analysis

In [20]:
import pandas as pd
from fcns import *

In [36]:
eustocks_df = pd.read_csv('datasets/eustocks.csv')
# freq = 'B' means 5 day week, no holidays
eustocks_df = pd.DataFrame(eustocks_df.values, columns=['0', 'DAX', 'SMI', 'CAC', 'FTSE'], index=pd.DatetimeIndex(start='1991-01-01', periods=1860, freq='B')).drop(columns=['0'])
#print(list(eustocks_df.columns))
eustocks_df.head()


C:\Users\darrahts\Anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,DAX,SMI,CAC,FTSE
1991-01-01,1628.75,1678.1,1772.8,2443.6
1991-01-02,1613.63,1688.5,1750.5,2460.2
1991-01-03,1606.51,1678.6,1718.0,2448.2
1991-01-04,1621.04,1684.1,1708.1,2470.4
1991-01-07,1618.16,1686.6,1723.1,2484.7


In [43]:
import functools
eustocks_df.apply(functools.partial(stationarity_test))

IndexError: ('index out of bounds', 'occurred at index DAX')